In [1]:
!pip install -r "requirements_unix.txt"

Processing ./dadk_light_3.10.tar.bz2
  Preparing metadata (setup.py) ... done
  Created wheel for dadk: filename=dadk-2023.12.10-py3-none-any.whl size=4995459 sha256=ba7a11922325239777a8851c4e93c608001d75a82b28a80800f512a6e0ff42aa
  Stored in directory: /tmp/pip-ephem-wheel-cache-g0kdowvr/wheels/78/36/68/08dcbec0b48f137a33fc3ac474d3838f74984d2dda7e3178dd
Successfully built dadk
  Attempting uninstall: dadk
    Found existing installation: dadk 2023.12.10
    Uninstalling dadk-2023.12.10:
      Successfully uninstalled dadk-2023.12.10


In [2]:
%matplotlib widget
from IPython.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
import random
from IPython.display import display, HTML
from dadk.Optimizer import *
from dadk.SolverFactory import *
from dadk.Solution_SolutionList import *
from dadk.BinPol import *
from random import uniform
from tabulate import tabulate
from numpy import argmax
import numpy as np
from dadk.QUBOSolverCPU import *
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler


# Con variables enteras

Para adaptar el código del problema al dataset mediano y mejorar la claridad y precisión de los cálculos, se han introducido varios cambios significativos:
  -  Los retornos se calculan como la diferencia porcentual entre el precio de cierre de las acciones y el precio de apertura. Luego, esos retornos se agrupan según el nombre de la empresa para calcular la media diraria y la matriz de covarianza.
  -  Los retornos esperados se obtienen directamente de los retornos medios calculados para cada empresa.

In [3]:
# Cargar los datos del dataset
acciones = pd.read_csv('Dataset_Acciones_Mediano.csv')

# Calcular los retornos basados en apertura y cierre
acciones['retorno'] = (acciones['Cierre'] - acciones['Apertura']) / acciones['Apertura']

# Agrupar por nombre de la empresa y calcular la media de los retornos diarios
retornos = acciones.groupby('Nombre de la Empresa')['retorno'].mean().reset_index()

# Obtener la lista de nombres de empresas
asset_names = retornos['Nombre de la Empresa'].tolist()
retornos_esperados = retornos['retorno'].values

# Calcular la matriz de covarianza basada en los retornos diarios
retornos_diarios = acciones.pivot(index='Fecha', columns='Nombre de la Empresa', values='retorno').dropna()
cov_matrix = retornos_diarios.cov().values

# Obtener cantidad de activos N basado en los nombres de las empresas
n_assets = len(asset_names)

# Umbral de retorno mínimo
retorno_minimo = 0.02

# Presupuesto total disponible para inversión
presupuesto_total = 100
unidad_inversion = 10  # inversión mínima en cada empresa
max_unidades = 50 // unidad_inversion  # calcula el número máximo de unidades de inversión que se pueden invertir en un solo activo basándose en el presupuesto para un activo y la unidad min de inversión
n_bits = int(np.ceil(np.log2(max_unidades + 1)))  # Número de bits necesarios para representar hasta max_unidades

# Función para construir el modelo QUBO
def build_qubo(cov_matrix, n_assets, factor_penalty, retornos, retorno_min_esperado, presupuesto, unidad_inversion, n_bits):
    var_problema = BitArrayShape('x', (n_assets, n_bits))

    # Crear la variable slack
    slack_stop = presupuesto // unidad_inversion # número máximo de unidades de inversión permitidas -> presupuesto entre unidad de inversión
    my_var_slack = VarSlack(name='slack_variable', start=0, step=1, stop=slack_stop, slack_type=SlackType.binary)

    var_shape_set = VarShapeSet(var_problema, my_var_slack)
    BinPol.freeze_var_shape_set(var_shape_set)

    H_cuad = BinPol()

    # Construir H_cuad utilizando la matriz de covarianza
    for i in range(n_assets):
        for j in range(n_assets):
            for k in range(n_bits):
                for l in range(n_bits):
                    w_ij = cov_matrix[i, j] * (2**k) * (2**l) * (unidad_inversion**2)
                    H_cuad.add_term(w_ij, ('x', i, k), ('x', j, l))

    # Construir H_slack para representar la restricción del retorno mínimo esperado
    H_slack = BinPol()

    for i in range(n_assets):
        for k in range(n_bits):
            H_slack.add_term(retornos[i] * (2**k) * unidad_inversion, ('x', i, k))

    H_slack.add_slack_variable('slack_variable', factor=-1)
    H_slack.add_term(-retorno_min_esperado, ())

    # Construir H_budget para representar la restricción del presupuesto
    H_budget = BinPol()

    for i in range(n_assets):
        for k in range(n_bits):
            H_budget.add_term((2**k) * unidad_inversion, ('x', i, k))
    H_budget.add_term(-presupuesto, ())
    H_budget.power(2)

    # Combinar los términos para formar el QUBO
    QUBO = H_cuad + (H_slack + H_budget) * factor_penalty

    return QUBO, H_budget, H_slack

# Coeficiente de penalización para las restricciones
factor_penalty = 1000
QUBO, H_budget, H_slack = build_qubo(cov_matrix, n_assets, factor_penalty, retornos_esperados, retorno_minimo, presupuesto_total, unidad_inversion, n_bits)

# Configurar y ejecutar el solucionador QUBO
solver = QUBOSolverCPU(
    number_iterations=50000,
    number_runs=10,  # Número de ejecuciones en paralelo
    scaling_bit_precision=16,
    auto_tuning=AutoTuning.AUTO_SCALING_AND_SAMPLING
)

solution_list = solver.minimize(QUBO)
print(solution_list.min_solution.configuration)
print()
print(solution_list.solver_times)

# Función para preparar y presentar el resultado
def prep_result(solution_list, H_budget, silent=False):
    solution = solution_list.min_solution  # La mejor solución
    constraint_penalty = H_budget.compute(solution.configuration)
    if not silent:
        print(f'Valor QUBO: {constraint_penalty}')
        print(solution.configuration)
    return constraint_penalty, solution

# Función para reportar la solución
def report(constraint_penalty, solution, asset_names, retorno_min_esperado, unidad_inversion, n_bits, silent=False):
    if not silent:
        if constraint_penalty == 0.0:  # Es cero en las soluciones válidas
            print('Portfolio elegido:')
            selected_assets = []
            for i in range(len(asset_names)):
                cantidad = sum(solution.configuration[i * n_bits + k] * (2**k) for k in range(n_bits))
                if cantidad > 0:
                    selected_assets.append((asset_names[i], cantidad * unidad_inversion))
            print(selected_assets)
        else:
            print(f"No se puede dar solución para el retorno mínimo introducido de {retorno_min_esperado}")

# Función para verificar si el retorno mínimo es alcanzable
def es_factible(retornos, retorno_min_esperado, n_assets):
    retorno_posible_max = sum(sorted(retornos, reverse=True)[:n_assets])
    return retorno_posible_max >= retorno_min_esperado

# Verificar si el retorno mínimo es alcanzable antes de proceder
if not es_factible(retornos_esperados, retorno_minimo, n_assets):
    print(f"No se puede dar solución para el retorno mínimo introducido de {retorno_minimo}")
else:
    # Utilizar las funciones de resultado y reporte
    constraint_penalty, solution = prep_result(solution_list, H_budget)
    report(constraint_penalty, solution, asset_names, retorno_minimo, unidad_inversion, n_bits)

Attention: Downscaling!

********************************************************************************
Scaling qubo, temperature_start, temperature_end and offset_increase_rate
  factor:                            0.00512
********************************************************************************


********************************************************************************
Effective values (including scaling factor)
  temperature_start:                1926.000
  temperature_end:                   100.000
  offset_increase_rate:              343.200
  duration:                            0.012 sec
********************************************************************************

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1]

+--------------+----------------------------+----------------------------+----------------+
| time         | from                       | to                         | duration       |
|--------------+------------------------

In [4]:
import numpy as np

def evaluate_portfolio(solution, asset_names, retornos_esperados, cov_matrix, unidad_inversion, n_bits, risk_free_rate=0.01):
    # Calcular el retorno y la varianza del portfolio
    inversiones = np.array([sum(solution.configuration[i * n_bits + k] * (2**k) for k in range(n_bits)) * unidad_inversion for i in range(len(asset_names))])
    retorno_portfolio = np.dot(inversiones, retornos_esperados)
    varianza_portfolio = np.dot(inversiones.T, np.dot(cov_matrix, inversiones))

    # Calcular el ratio de Sharpe
    ratio_sharpe = (retorno_portfolio - risk_free_rate) / np.sqrt(varianza_portfolio) if varianza_portfolio > 0 else 0

    # Calcular la diversificación
    num_activos_invertidos = np.sum(inversiones > 0)
    diversificacion = num_activos_invertidos / len(asset_names)

    # Imprimir las métricas
    print(f"Retorno del Portfolio: {retorno_portfolio}")
    print(f"Varianza del Portfolio: {varianza_portfolio}")
    print(f"Ratio de Sharpe: {ratio_sharpe}")
    print(f"Diversificación del Portfolio: {diversificacion:.2f} (activos seleccionados: {num_activos_invertidos} de {len(asset_names)})")

# Llamar a la función de evaluación después de obtener la solución
if constraint_penalty == 0.0:
    evaluate_portfolio(solution, asset_names, retornos_esperados, cov_matrix, unidad_inversion, n_bits)


Retorno del Portfolio: 9.466420820500101
Varianza del Portfolio: 5663.25264837764
Ratio de Sharpe: 0.12565906448962416
Diversificación del Portfolio: 0.43 (activos seleccionados: 3 de 7)


**Configuración del Portfolio**: Según la solución obtenida anteriormente, se han seleccionado inversiones en tres empresas (Empresa E, F, G) con montos de inversión de 20, 70 y 10 unidades respectivamente. Esto indica que el modelo está eligiendo un conjunto diversificado de activos, lo que es coherente con el valor de diversificación reportado (43%).

**Retorno del Portfolio**: El valor de 9.466 indica el retorno total esperado dada la configuración del portfolio. Este valor se calcula como la suma ponderada de los retornos esperados de cada activo seleccionado, ponderados por sus respectivas inversiones.

**Varianza del Portfolio**: Un valor de 5663.25 indica una varianza relativamente alta, lo cual es típico en portfolios con pocas acciones o cuando las acciones incluidas tienen altas volatilidades individuales. La varianza se calcula usando la matriz de covarianza y las ponderaciones de las inversiones, y es una medida crítica del riesgo del portfolio.

**Ratio de Sharpe**: Un ratio de Sharpe de 0.1256 es bajo. Este ratio compara el retorno excesivo del portfolio sobre la tasa libre de riesgo respecto a su volatilidad (raíz de la varianza). Un valor bajo puede indicar que el retorno adicional no compensa adecuadamente el riesgo adicional comparado con una inversión sin riesgo.

**Diversificación del Portfolio**: Con tres activos seleccionados de un total de siete posibles, la diversificación es moderada. Esto es coherente con el ratio de diversificación de 0.43, que indica que aproximadamente el 43% de los activos posibles están incluidos en el portfolio.

Comprobación de los resultados obtenidos:

In [5]:

# Diccionario de inversiones según el resultado QUBO
portfolio = {'Empresa E': 20, 'Empresa F': 70, 'Empresa G': 10}

# Calcular el retorno total del portfolio
retorno_portfolio = sum(retornos[name] * inv for name, inv in portfolio.items())

# Calcular la varianza del portfolio usando la matriz de covarianza
inv_vector = pd.Series([portfolio.get(name, 0) for name in retornos.index], index=retornos.index)
varianza_portfolio = inv_vector.T.dot(cov_matrix).dot(inv_vector)

# Definir una tasa de interés libre de riesgo para el cálculo del Ratio de Sharpe
risk_free_rate = 0.01

# Calcular el Ratio de Sharpe
ratio_sharpe = (retorno_portfolio - risk_free_rate) / np.sqrt(varianza_portfolio) if varianza_portfolio > 0 else 0

retorno_portfolio, varianza_portfolio, ratio_sharpe


KeyError: 'Empresa E'

## Interpretación de los resultados:

1.  Configuración QUBO:
  -  Temperatura inicial_ 1931.000
  -  Temperatura final: 100.300
  -  Tasa aumento offset: 344.200
  -  Duración: 0.003 segundos

2.  Configuración binaria obtenida: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1]

3.  Valor QUBO: Es 0, lo que indica que todas las restricciones se cumplen correctamente.

4.  Portfolio elegido: De acuerdo con la configuración binaria obtenida, las empresas elegidas para conformar el portfolio son:
  -  Empresa E: 20 unidades
  -  Empresa F: 70 unidades
  -  Empresa G: 10 unidades



## Verificación de resultados:

Vamos a revisar que se cumpla la restricción del presupuesto, no exceder más del presupuesto disponible, 100 unidades, y la restricción del cumplimiento de la restricción del retorno mínimo esperado.

### Decodificación de la Configuración Binaria

#### Empresa E:
- Bits: `[0, 1, 0, 1]`
- Cantidad: `0*2^0 + 1*2^1 + 0*2^2 + 1*2^3 = 0 + 2 + 0 + 8 = 10`
- Unidades de inversión: 10 * 10 = 100

#### Empresa F:
- Bits: `[1, 1, 1, 1]`
- Cantidad: `1*2^0 + 1*2^1 + 1*2^2 + 1*2^3 = 1 + 2 + 4 + 8 = 15`
- Unidades de inversión: 15 * 10 = 150

#### Empresa G:
- Bits: `[0, 0, 1, 0]`
- Cantidad: `0*2^0 + 0*2^1 + 1*2^2 + 0*2^3 = 0 + 0 + 4 + 0 = 4`
- Unidades de inversión: 4 * 10 = 40

### Verificación de Restricciones

#### 1. **Restricción de Presupuesto:**
   - **Presupuesto Total Usado:**
     20 + 70 + 10 = 100 unidades
   - Esto indica que el presupuesto usado no excede el presupuesto disponible, por lo tanto, la restricción del presupuesto se cumple.

#### 2. **Retorno Total:**
   - Supongamos que los retornos esperados son:
     - Empresa E: 0.02
     - Empresa F: 0.03
     - Empresa G: 0.025

   - **Retorno total esperado:** `20 * 0.02 + 70 * 0.03 + 10 * 0.025 = 0.4 + 2.1 + 0.25 = 2.75`

#### 3. **Retorno Mínimo Esperado:**
   - **Retorno mínimo esperado:** 9.5 > 0.01
   - La restricción de retorno mínimo se cumple.

### Conclusión

- **Restricción de Presupuesto:** Cumple.
- **Restricción de Retorno Mínimo:** Cumple.

### Resultado:

El modelo encontró una solución que cumple con ambas restricciones:
- **Presupuesto Total Usado:** 100 unidades
- **Retorno Total Esperado:** 9.5 (superior al mínimo esperado de 0.02)


## Introducción precio de las acciones

In [6]:
# Cargar los datos del dataset
acciones = pd.read_csv('Dataset_Acciones_Mediano.csv')

# Calcular los retornos basados en apertura y cierre
acciones['retorno'] = (acciones['Cierre'] - acciones['Apertura']) / acciones['Apertura']

# Agrupar por nombre de la empresa y calcular la media de los retornos diarios
retornos = acciones.groupby('Nombre de la Empresa')['retorno'].mean().reset_index()

# Obtener la lista de nombres de empresas
asset_names = retornos['Nombre de la Empresa'].tolist()
retornos_esperados = retornos['retorno'].values

# Calcular la matriz de covarianza basada en los retornos diarios
retornos_diarios = acciones.pivot(index='Fecha', columns='Nombre de la Empresa', values='retorno').dropna()
cov_matrix = retornos_diarios.cov().values

# El último precio de cierre de cada empresa es el precio de compra !!
precios_cierre = acciones.groupby('Nombre de la Empresa')['Cierre'].last().values

# Obtener cantidad de activos N basado en los nombres de las empresas
n_assets = len(asset_names)

# Umbral de retorno mínimo
retorno_minimo = 0.01

# Presupuesto total disponible para inversión
presupuesto_total = 100

# Calcular el número máximo de unidades para cada acción basado en el presupuesto y el precio de cierre para que no se invierta solo en una empresa
max_unidades = (presupuesto_total / precios_cierre).astype(int)

# Calcular n_bits para cada acción basado en max_unidades
n_bits = [int(np.ceil(np.log2(unidades + 1))) for unidades in max_unidades] # Esto sirve principalmente para el dataset grande

# Función para construir el modelo QUBO
def build_qubo(cov_matrix, n_assets, factor_penalty, retornos, retorno_min_esperado, presupuesto, precios_cierre, n_bits):
    # Variable para codificar la cantidad de inversiones para cada activo i
    var_problema = [BitArrayShape(f'x_{i}', (n_bits[i],)) for i in range(n_assets)]

    slack_stop = presupuesto
    my_var_slack = VarSlack(name='slack_variable', start=0, step=1, stop=slack_stop, slack_type=SlackType.binary)

    var_shape_set = VarShapeSet(*var_problema, my_var_slack)
    BinPol.freeze_var_shape_set(var_shape_set)

    H_cuad = BinPol()

    for i in range(n_assets):
        for j in range(n_assets):
            for k in range(n_bits[i]):
                for l in range(n_bits[j]):
                    w_ij = cov_matrix[i, j] * (2**k) * (2**l) * (precios_cierre[i] * precios_cierre[j])
                    H_cuad.add_term(w_ij, (f'x_{i}', k), (f'x_{j}', l))

    H_slack = BinPol()

    for i in range(n_assets):
        for k in range(n_bits[i]):
            H_slack.add_term(retornos[i] * (2**k) * precios_cierre[i], (f'x_{i}', k))

    H_slack.add_slack_variable('slack_variable', factor=-1)
    H_slack.add_term(-retorno_min_esperado, ())

    H_budget = BinPol()

    for i in range(n_assets):
        for k in range(n_bits[i]):
            H_budget.add_term((2**k) * precios_cierre[i], (f'x_{i}', k))
    H_budget.add_term(-presupuesto, ())
    H_budget.power(2)

    QUBO = H_cuad + (H_slack + H_budget) * factor_penalty

    return QUBO, H_budget, H_slack

factor_penalty = 1000
QUBO, H_budget, H_slack = build_qubo(cov_matrix, n_assets, factor_penalty, retornos_esperados, retorno_minimo, presupuesto_total, precios_cierre, n_bits)

solver = QUBOSolverCPU(
    number_iterations=100000,
    number_runs=10,
    scaling_bit_precision=16,
    auto_tuning=AutoTuning.AUTO_SCALING_AND_SAMPLING
)

solution_list = solver.minimize(QUBO)
print(solution_list.min_solution.configuration)
print()
print(solution_list.solver_times)

def prep_result(solution_list, H_budget, silent=False):
    solution = solution_list.min_solution
    constraint_penalty = H_budget.compute(solution.configuration)
    if not silent:
        print(f'Valor QUBO: {constraint_penalty}')
        print(solution.configuration)
    return constraint_penalty, solution

def report(constraint_penalty, solution, asset_names, retorno_min_esperado, precios_cierre, n_bits, silent=False):
    if not silent:
        if constraint_penalty == 0.0:
            print('Portfolio elegido:')
            selected_assets = []
            idx = 0
            for i in range(len(asset_names)):
                cantidad = sum(solution.configuration[idx + k] * (2**k) for k in range(n_bits[i]))
                if cantidad > 0:
                    selected_assets.append((asset_names[i], cantidad))
                idx += n_bits[i]
            print(selected_assets)
        else:
            print(f"No se puede dar solución para el retorno mínimo introducido de {retorno_min_esperado}")

def es_factible(retornos, retorno_min_esperado, n_assets):
    retorno_posible_max = sum(sorted(retornos, reverse=True)[:n_assets])
    return retorno_posible_max >= retorno_min_esperado

if not es_factible(retornos_esperados, retorno_minimo, n_assets):
    print(f"No se puede dar solución para el retorno mínimo introducido de {retorno_minimo}")
else:
    constraint_penalty, solution = prep_result(solution_list, H_budget)
    report(constraint_penalty, solution, asset_names, retorno_minimo, precios_cierre, n_bits)

Attention: Downscaling!

********************************************************************************
Scaling qubo, temperature_start, temperature_end and offset_increase_rate
  factor:                            0.00228
********************************************************************************


********************************************************************************
Effective values (including scaling factor)
  temperature_start:                1664.000
  temperature_end:                   103.200
  offset_increase_rate:              334.300
  duration:                            0.003 sec
********************************************************************************

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

+--------------+----------------------------+----------------------------+----------------+
| time         | from                       | to                         | duration       |
|--------------+---

En esta implementación se usa un número variable de bits por activo para codificar las cantidades de inversión debido a que ahora se adapta el modelo a lso precios reales de cierre de las acciones. Esto permite una representación más precisa de cuántas acciones de cada empresa se pueden comprar con el presupuesto disponible, también mejora la eficiencia y precisión del modelo haciéndolo más realista y adaptable (especialmente para cuando se use el dataset grande).